# E2E Approach -- Scenario II

In this notebook, we report the code related to the *hybrid* appraoch in our paper [[1]](#ourpaper). We consider the delay caused by the feedback loop used by the mobile user equipment to send to the base station the channel state information (CSI). Due to this feedback delay, the CSI available at the base station becomes outdated. In our paper, we present an E2E appraoch, which takes in input the channel history available at the base station and maps it to the probability of an error event for all the modulation and coding schemes (MCSs) the base station can select. The error event corresponds to an unsuccessfully decoded frame at the receiver. If the error event probabilities are known for all the MCSs, then it is straightforward to select the MCS that maximizes the spectral efficiency.

Under Scenario II, we build a differnt neural network for each delay, but we train it to work on the full range of dopplers and signal-to-noise ratios.

It must be noted that the training datasets listed below in the code are currently not available in the repository due to space limitations. The **training datasets can be found at**: https://kth.box.com/s/tcd7y7rg3yau75kctw3regmyns8kfkr6 in the folder *Datasets*. At any rate, in the repository, the reader can also find the codes in *radio_data* folder which can be run to generate the datasets. 

**Note**: the training might take some hours, depending on the available computational resources, the dimension of the training set, the dimension of the network, and the number of epochs. 

<a id='ourpaper'></a> [1] "Wireless link adaptation - a hybrid data-driven and model-based approach", Lissy Pellaco, Vidit Saxena, Mats Bengtsson, Joakim Jaldén. Submitted to SPAWC 2020.

# Import libraries and utility functions

In [ ]:
import numpy as np
import utilities as utils
from keras.optimizers import Adam
from keras.backend.tensorflow_backend import set_session
from keras.backend import clear_session
import tensorflow as tf
import time

In [ ]:
# Maximum feedback delay that we consider
maximum_delay = 9

# Number of past CSI stored at the base station and used as input to the neural network
ANN_MEMORY = 10
# Flag used to indicate if the channel is noisy
CHANNEL_EST_NOISE = True

# Parameters related to neural network training
TRAINING_FRACTION = 0.2
BATCH_SIZE = 32
NROF_EPOCS = 20 

# Flag to indicate if the trained models should be saved
save_model = False

## Load the Dataset

The channel dataset is a dict with the following keys :  
 - 'channel'
     - Complex channel coefficients 
     - Numpy array [ NROF_FRAMES x NROF_SUBCARRIERS x NROF_SNRS]
 - 'block_success'
      - Binary success events (ACKs)
      - Numpy array [ NROF_FRAMES x NROF_MCS x NROF_SNRS]
 - 'snrs_db '      
     - Evaluated average SNR values
     - Numpy array [ NROF_SNRS ]
 - 'block_sizes'
     - Evaluated transport block sizes
     - Numpy array [ NROF_MCS ]
     
The name of the dataset, e.g., ITU_VEHICULAR_B_5000_60kmph, is to be interpreted in this way: 
 - channel model (ITU_VEHICULAR_B)
 - number of channel realizations (5000)
 - relative velocity between the base station and the user mobile equipment (60kmph)
 
The **training datasets can be found at**: https://kth.box.com/s/tcd7y7rg3yau75kctw3regmyns8kfkr6 in the folder *Datasets*

In [ ]:
# The files stored in the file_set ARE NOT in the repository due to space limitations.
# The training datasets can be found at: https://kth.box.com/s/tcd7y7rg3yau75kctw3regmyns8kfkr6 in the folder *Datasets*
# The reader has also access to the "radio_data/Generate_Data.ipynb" which we used to generate the training datasets.
# N.B. a small training set with new samples at snr of 5dB is added to boost the performance at low snr
FADING_CHANNEL_DATAFILES = []

FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_30kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_45kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_60kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_75kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_90kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_105kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_120kmph.npy')

# Data Preprocessing

In [ ]:
channel_coeff  = []
block_success  = []

for file in FADING_CHANNEL_DATAFILES:
    DATASET = np.load( file, allow_pickle = True )[()]
    
    nrof_train_samples = int( TRAINING_FRACTION * DATASET['channel'].shape[0] )
    coeff = utils.calculate_channel_coefficients_scaled( DATASET['channel'][ :nrof_train_samples, :, : ],
                                                         DATASET['snrs_db'],
                                                         channel_estimation_noise = CHANNEL_EST_NOISE )
        
    channel_coeff.append( coeff )
    block_success.append( DATASET['block_success'][ :nrof_train_samples, :, : ] )
    
channel_coeff = np.vstack( channel_coeff )
block_success = np.vstack( block_success )

NROF_FRAMES, NROF_SUBCARRIERS, NROF_SNRS = channel_coeff.shape
NROF_MCS = block_success.shape[ 1 ]

channel_coeff_concat = np.concatenate( ( np.real( channel_coeff ), np.imag( channel_coeff ) ), axis = 1 )


## Build the Neural Network model, define the optimizer, and the cost function

In [ ]:
def create_ann_model():
    from keras.models import Sequential
    from keras.layers import Dense, Dropout

    model = Sequential()
    model.add( Dense( 1024, 
                      input_dim = NROF_SUBCARRIERS * ANN_MEMORY * 2, 
                      kernel_initializer='normal', 
                      activation='relu' ) )

    model.add( Dense( 512, 
                      kernel_initializer = 'normal', 
                      activation='relu' ) )

    model.add( Dense( 1024, 
                      kernel_initializer = 'normal', 
                      activation='relu' ) )
    
    model.add( Dense( NROF_MCS, 
                      kernel_initializer='normal', 
                      activation='sigmoid' ) )

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # for binary classification

    
    return model

## Train the Neural Network

In [ ]:
for delay in range(0,maximum_delay + 1):
    
    channel_coeff  = []
    block_success  = []

    # Extract training data from all datasets, excluding the last one (used later)
    for file in FADING_CHANNEL_DATAFILES[:-1]:
        DATASET = np.load( file, allow_pickle = True )[()]

        nrof_train_samples = int( TRAINING_FRACTION * DATASET['channel'].shape[0] )
        coeff = utils.calculate_channel_coefficients_scaled( DATASET['channel'][ :nrof_train_samples, :, : ],
                                                             DATASET['snrs_db'],
                                                             channel_estimation_noise = CHANNEL_EST_NOISE )

        channel_coeff.append( coeff )
        block_success.append( DATASET['block_success'][ :nrof_train_samples, :, : ] )
    
    channel_coeff = np.vstack( channel_coeff )
    block_success = np.vstack( block_success )

    NROF_FRAMES, NROF_SUBCARRIERS, NROF_SNRS = channel_coeff.shape
    NROF_MCS = block_success.shape[ 1 ]

    channel_coeff_concat = np.concatenate( ( np.real( channel_coeff ), np.imag( channel_coeff ) ), axis = 1 )

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True  

    sess = tf.Session( config = config )
    set_session(sess) 
    if delay > 0:
        train_input  = utils.flatten_snr_axis( channel_coeff_concat[ :-delay, :, :] )
    else:
        train_input  = utils.flatten_snr_axis( channel_coeff_concat )

    train_target = utils.flatten_snr_axis( block_success [ delay :, :, : ] )

    train_input, train_target = utils.shuffle_data( train_input, train_target )

    model = create_ann_model( )
    
    # Storing current time 
    start = time.time()

    history = model.fit( utils.stack_features( train_input, ANN_MEMORY ), 
                         train_target, 
                         batch_size = BATCH_SIZE, 
                         epochs     = NROF_EPOCS, 
                         validation_split = 0.1, 
                         verbose    = 1 ) # the "verbose parameter" can be changed to display more about the training progess of each epoch
    channel_coeff  = []
    block_success  = []

    # Extract training data from last dataset
    for file in [FADING_CHANNEL_DATAFILES[-1]]:
        DATASET = np.load( file, allow_pickle = True )[()]

        nrof_train_samples = int( TRAINING_FRACTION * DATASET['channel'].shape[0] )
        coeff = utils.calculate_channel_coefficients_scaled( DATASET['channel'][ :nrof_train_samples, :, : ],
                                                             DATASET['snrs_db'],
                                                             channel_estimation_noise = CHANNEL_EST_NOISE )

        channel_coeff.append( coeff )
        block_success.append( DATASET['block_success'][ :nrof_train_samples, :, : ] )
    
    channel_coeff = np.vstack( channel_coeff )
    block_success = np.vstack( block_success )

    channel_coeff_concat = np.concatenate( ( np.real( channel_coeff ), np.imag( channel_coeff ) ), axis = 1 )

    if delay > 0:
        train_input  = utils.flatten_snr_axis( channel_coeff_concat[ :-delay, :, :] )
    else:
        train_input  = utils.flatten_snr_axis( channel_coeff_concat )

    train_target = utils.flatten_snr_axis( block_success [ delay :, :, : ] )

    train_input, train_target = utils.shuffle_data( train_input, train_target )
    
    # Additional training
    history = model.fit( utils.stack_features( train_input, ANN_MEMORY ), 
                         train_target, 
                         batch_size = BATCH_SIZE, 
                         epochs     = NROF_EPOCS, 
                         validation_split = 0.1, 
                         verbose    = 1 ) # the "verbose parameter" can be changed to display more about the training progess of each epoch

        
    file = 'Trained_models_ScenarioII/ANN_E2E_MCS_PRED_DELAY_%d_MEM_10_SNR.h5'%( delay )
    print("Training the neural network took:",time.time()-start)
    if save_model == True:
        model.save( file )
        print( 'Saved model to %s'%( file ) )
